In [1]:
# !pip install transformers

In [12]:
import pandas as pd
import numpy as np
# import glob
# import re

# from tokenizers import SentencePieceBPETokenizer, normalizers, decoders
# from tokenizers.normalizers import Replace

# from nltk.tokenize import word_tokenize

from transformers import AutoConfig, AutoModelForCausalLM
from transformers import PreTrainedTokenizerFast
import torch

In [2]:
config = AutoConfig.from_pretrained("openai-community/gpt2")

config.n_head = 6
config.n_layer = 6
config.vocab_size = 32768
config.n_embd = 384
config.n_ctx = 768

model = AutoModelForCausalLM.from_config(config=config)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 23623680


In [3]:
# import zipfile

# # Replace 'path/to/your/file.zip' with your actual zip file path
# with zipfile.ZipFile('Archive 2.zip', 'r') as zip_ref:
#     zip_ref.extractall('train')  # Specify the directory to extract to

In [5]:
# X, Y = torch.load('train/X_train.pt'), torch.load("train/Y_train.pt")

In [6]:
# X.shape

In [9]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
device

'mps'

In [10]:
model = model.to(device)

In [11]:
# X = X[:6398519]
# Y = Y[:6398519]

In [12]:
# from torch.optim import AdamW
# import torch
# import numpy as np
# import os

# # Define the path for saving model checkpoints
# checkpoint_dir = './model_checkpoints'
# os.makedirs(checkpoint_dir, exist_ok=True)

# optim = AdamW(model.parameters(), lr=0.01)
# batch_size = 1024
# train_size = len(X)
# logging_interval = 0.1
# total_loss = 0

# perplexity_list = np.array([])
# final_loss = np.array([])
# steps_per_epochs = len(range(0, train_size, batch_size))
# logging_step = int(steps_per_epochs * logging_interval)

# for epoch in range(5):
#     epoch_loss = np.array([])
#     model.train()  # Ensure the model is in training mode

#     for idx, iter in enumerate(range(0, train_size, batch_size)):
#         x = X[iter: iter + batch_size]
#         y = Y[iter: iter + batch_size]
#         x, y = x.to(device), y.to(device)

#         # Ensure y is a 1D tensor
#         y = y.squeeze()

#         # Forward pass
#         y_pred = model(x)
#         loss = torch.nn.functional.cross_entropy(y_pred.logits[:, -1, :], y)

#         # Accumulate the loss for logging
#         total_loss += loss.item()

#         # Zero out gradients, backpropagate, and update weights
#         optim.zero_grad()
#         loss.backward()
#         optim.step()

#         # Track epoch loss
#         epoch_loss = np.append(epoch_loss, loss.item())

#         # Log perplexity at specified intervals
#         if idx % logging_step == 0 and idx != 0:
#             avg_loss = total_loss / ((iter // batch_size) + 1)
#             perplexity = torch.exp(torch.tensor(avg_loss))
#             perplexity_list = np.append(perplexity_list, perplexity)
#             print(f"Perplexity at epoch {epoch + 1}, step {iter}: {perplexity.item()}")

#     # Save checkpoint after each epoch
#     checkpoint_path = os.path.join(checkpoint_dir, f'model_epoch_{epoch + 1}.pth')
#     torch.save({
#         'epoch': epoch + 1,
#         'model_state_dict': model.state_dict(),
#         'optimizer_state_dict': optim.state_dict(),
#         'loss': epoch_loss.mean()
#     }, checkpoint_path)
#     print(f"Checkpoint saved at {checkpoint_path}")

#     print(f"Loss of epoch {epoch + 1}: {epoch_loss.mean()}")
#     total_loss = 0  # Reset total_loss after each epoch


### Test the model


In [13]:
config = AutoConfig.from_pretrained("openai-community/gpt2")

config.n_head = 6
config.n_layer = 6
config.vocab_size = 32768
config.n_embd = 384
config.n_ctx = 768

model = AutoModelForCausalLM.from_config(config=config)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 23623680


In [14]:
from tokenizers import SentencePieceBPETokenizer

checkpoint_path = './Model/model_epoch_5.pth'

model.load_state_dict(torch.load(checkpoint_path,map_location=torch.device('mps'))['model_state_dict'])

# Set the model to evaluation mode
model.eval()

# Move the model to the device (CPU or GPU)
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
model.to(device)


tokenizer = SentencePieceBPETokenizer(
    "../Tokenizer/my_tokenizer_1-vocab.json",
    "../Tokenizer/my_tokenizer_1-merges.txt"
)


/var/folders/4k/xxxy9tg127s6hyrvg1srwct00000gn/T/ipykernel_54602/2477161886.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_p

In [ ]:

# Example test input (should be tokenized and preprocessed)
test_input = "તમારા ઘર માટે યુગાદી શણગાર"
encoded_input = tokenizer.encode(test_input)
encoded_input.ids

[622, 1531, 185, 4003, 3503, 10801]

In [6]:
input_ids = torch.Tensor(encoded_input.ids).to(device='mps', dtype=torch.long)
input_ids

tensor([  622,  1531,   185,  4003,  3503, 10801], device='mps:0')

In [7]:
with torch.no_grad():
    try:
        output = model(input_ids)
    except Exception as e:
        print(f"Error during generation: {e}")
        exit()


In [8]:
logits = output.logits
predicted_token_ids = torch.argmax(logits, dim=-1)

In [9]:
tokenizer.decode(predicted_token_ids.tolist())

'જ પર પરરનીરની '

In [ ]:
# Assuming `logits` is the output from the model after running inference
# and contains logits for the current sequence.
logits = output.logits

# Step 1: Extract the logits for the last token in the sequence
current_input_ids = input_ids[0].tolist()  # Start with the original input

# Step 2: Loop to generate 10 tokens sequentially
input = "તમારા ઘર માટે યુગાદી શણગાર"
output_token = ""
for _ in range(10):
    input = torch.Tensor(tokenizer.encode(input + output_token).ids).to(device, dtype = torch.long)
    with torch.no_grad():
        output = model(input)
        logits = output.logits
        

AttributeError: 'int' object has no attribute 'append'